88. 合并两个有序数组 

https://leetcode.cn/problems/merge-sorted-array/description/?envType=study-plan-v2&envId=top-interview-150

In [1]:
from typing import List

class Solution:
    def merge(self, nums1: List[int], m: int, nums2: List[int], n: int) -> None:
        """
        Do not return anything, modify nums1 in-place instead.
        """
        assert len(nums1) == m + n
        assert len(nums2) == n

        index = 0
        for num in nums2:
            if nums1[index] == 0:
                index += 1
            else:
                while num >= nums1[index] and nums1[index] != 0:
                    index += 1

            nums1 = nums1[:index] + [num] + nums1[index:-1]
        return nums1